In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob

results = glob('./results/*.html')

In [2]:
data = pd.DataFrame()

for result_file in results:

    sex = 'M' if '_M_' in result_file else 'W'

    with open(result_file, 'r', encoding="utf8") as file:
        html = file.read()

    parsed_html = BeautifulSoup(html, 'html.parser')

    elements = ( parsed_html.find_all(class_ = 'list-active list-group-item row') +
                    parsed_html.find_all(class_ = 'list-group-item row') )

    # names = []
    places = []
    startnumbers = []
    agegroups = []
    times = []

    for element in elements:
        # names.append(element.find(class_ = 'list-field type-fullname').text)
        places.append(element.find(class_ = 'list-field type-place place-primary numeric').text)
        startnumbers.append(element.find(class_ = 'list-field type-field').text.replace('Bib Number', '').strip())
        agegroups.append(element.find(class_ = 'list-field type-age_class').text.replace('Age Group', '').strip())
        times.append(element.find(class_ = 'list-field type-time').text.replace('Net time', '').strip())

    temp_data = pd.DataFrame({
        'place': places,
        'number': startnumbers,
        'age': agegroups,
        'sex': [sex] * len(places),
        'time': times
    })

    data = pd.concat([data, temp_data], ignore_index=True)

data

,place,number,age,sex,time
0,1,5,H,M,02:06:16
1,3,2,H,M,02:07:02
2,5,16,H,M,02:07:47
3,7,36,H,M,02:09:35
4,9,9,30,M,02:09:43
...,...,...,...,...,...
12318,2818,F536,50,W,06:20:54
12319,2820,F1563,60,W,06:23:46
12320,2822,F878,H,W,06:26:18
12321,2824,F3992,H,W,06:34:13


In [3]:
data.to_csv('marathon_results.csv', index=False)